# CXR14 EDA

_"You can observe a lot by just watching." ~Yogi Berra_

This is an exploratory data analysis of the Chest X-ray CXR-14 dataset from the NIHCC.

**Contents**
1. [Setup and load data](#1.-Setup)
2. [Data cleaning](#2.-Data-cleaning)
3. [Begin EDA](#3.-Begin-EDA)
4. Explore Patient Age, Gender, and X-ray Follow-up #
5. Explore Finding Labels (target variable for modeling)
    - Distributions of 14 thoracic disease classes
    - Normal vs. non-normal
    - Pairwise correlations of findings
    - *Related* Co-morbidities for multi-label X-rays (pie chart [example])
6. Explore X-ray image size and resolution
7. *To Do:* X-ray image examples and augmentation
8. (optional) *To Do:* Binary logistic regression for likelihood of non-normal based on Age, Gender, and Follow-up #


## 1. Setup

### Load Python packages used for analysis

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualizing the data
import seaborn as sns # prettier visuals of data, built on matplotlib
from collections import OrderedDict # to preserve column order when creating DFs
import matplotlib.gridspec as gridspec
import matplotlib.ticker as ticker # for customizing axis tick marks in plots

In [2]:
sns.set_style('whitegrid')
%matplotlib inline

In [3]:
# Uncomment to check pandas environment
# pd.show_versions() # show versions of pandas and all of its dependencies
# pd.__version__ # show pandas version only

### Load data
Source: NIH  
File: Data_Entry_2017.csv  
Repo: https://nihcc.app.box.com/v/ChestXray-NIHCC/folder/36938765345  

In [4]:
xrayDataIn = pd.read_csv('../data/Data_Entry_2017.csv')

In [5]:
xrayDataIn.shape

(112120, 12)

**How many rows and columns are the data?**

The original X-ray data file provides 112,120 rows and 12 columns.

In [6]:
xrayDataIn.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


**What does each row of data represent?**

Each row is a single X-ray, each with the following information:

- Image Index - unique image PNG filename for each X-ray
- Finding Labels - pipe-separated list of pathologies detected in X-ray, or "No Finding" if none
- Follow-up # - zero for first X-ray per patient, and incremented by one for subsequent X-rays
- Patient Age - age, in years
- Patient ID
- Patient Gender - Category with two classes:
    - F for Female
    - M for Male
- View Position - Category with two classes:
    - PA for posterioranterior position
    - AP for anteroposterior position
- OriginalImage\[Width - image width, in pixels
- Height\] - image height, in pixels
- OriginalImagePixelSpacing\[x - horizontal image pixel spacing
- y\] - vertical image pixel spacing
- Unnamed: 11 - junk

In [7]:
# list each column by name, non-null counts, and data type
xrayDataIn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112120 entries, 0 to 112119
Data columns (total 12 columns):
Image Index                    112120 non-null object
Finding Labels                 112120 non-null object
Follow-up #                    112120 non-null int64
Patient ID                     112120 non-null int64
Patient Age                    112120 non-null int64
Patient Gender                 112120 non-null object
View Position                  112120 non-null object
OriginalImage[Width            112120 non-null int64
Height]                        112120 non-null int64
OriginalImagePixelSpacing[x    112120 non-null float64
y]                             112120 non-null float64
Unnamed: 11                    0 non-null float64
dtypes: float64(3), int64(5), object(4)
memory usage: 10.3+ MB


Observations:
- Total of 112,120 rows, one per X-ray
- No missing data, with exception of empty column 'Unnamed: 11'
- Categories 'Patient Gender' and 'View Position' imported with type *object*; change to type *category*

## 2. Data cleaning

The following clean-up supports subsequent EDA:
- Copy X-ray data set and drop unused column
- Clean up data types and column names
- Parse "Finding Labels" into separate columns for each pathology
- Create column to show sum of total pathologies found for each X-ray
- Create column with binary flag if Non-normal X-ray
- Create column with Age Groups for summarization

In [8]:
# copy dataframe and drop unused column
xrays = xrayDataIn.drop('Unnamed: 11', axis=1)

In [9]:
# change data type to category
for col in ['Patient Gender','View Position']:
    xrays[col] = xrays[col].astype('category')

# simplify column names
xrays.rename(columns = {'OriginalImage[Width':'ImgWidth'}, inplace = True)
xrays.rename(columns = {'Height]':'ImgHeight'}, inplace = True)
xrays.rename(columns = {'OriginalImagePixelSpacing[x':'ImgPxSpaceX'}, inplace = True)
xrays.rename(columns = {'y]':'ImgPxSpaceY'}, inplace = True)

In [10]:
# create new column for each disease
pathology_list = ['Cardiomegaly','Emphysema','Effusion','Hernia','Nodule','Pneumothorax','Atelectasis',
                  'Pleural_Thickening','Mass','Edema','Consolidation','Infiltration','Fibrosis','Pneumonia']

# for each pathology, for each row, set to 1 if label is found, or 0 if not found
for pathology in pathology_list :
    xrays.loc[:, pathology] = xrays['Finding Labels'].apply(lambda x: 1 if pathology in x else 0)

# Code block credited to Stephane Bernadec's Kaggle kernel.

In [11]:
# create column for total number of pathologies in each X-ray; axis=1 to sum by columns
xrays.loc[:, 'TotalPathos'] = xrays.iloc[:, 11:26].sum(axis=1)

# create column with binary flag if Non-normal X-ray
xrays.loc[:, 'Non-normal'] = xrays['TotalPathos'].apply(lambda x: 1 if (x > 0) else 0)

# create column with combined image width by height string
xrays.loc[: ,"ImageWxH"] = xrays["ImgWidth"].map(str) + "x" + xrays["ImgHeight"].map(str)

In [12]:
# create column with age groups
def age_groups(series):
    if series < 18:
        return "1-18 yrs"
    elif 18 <= series < 25:
        return "18-24 yrs"
    elif 25 <= series < 35:
        return "25-34 yrs"
    elif 35 <= series < 45:
        return "35-44 yrs"
    elif 45 <= series < 55:
        return "45-54 yrs"
    elif 55 <= series < 65:
        return "55-64 yrs"
    elif 65 <= series < 75:
        return "65-74 yrs"
    elif 75 <= series < 85:
        return "75-84 yrs"
    elif 85 <= series < 95:
        return "85-94 yrs"
    elif 95 <= series:
        return "95+ yrs"

xrays['Age Group'] = xrays['Patient Age'].apply(age_groups)

In [ ]:
xrays.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,ImgWidth,ImgHeight,ImgPxSpaceX,...,Mass,Edema,Consolidation,Infiltration,Fibrosis,Pneumonia,TotalPathos,Non-normal,ImageWxH,Age Group
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,...,0,0,0,0,0,0,1,1,2682x2749,55-64 yrs
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,0,0,0,0,0,0,2,1,2894x2729,55-64 yrs
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0,0,0,0,0,0,2,1,2500x2048,55-64 yrs
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,...,0,0,0,0,0,0,0,0,2500x2048,75-84 yrs
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,...,0,0,0,0,0,0,1,1,2582x2991,75-84 yrs


### Save cleaned data to CSV

In [ ]:
# save cleaned dataframe to CSV, ignoring the index
xrays.to_csv('../data/xrays-eda-clean.csv', index=False)

### Read cleaned data from CSV

In [ ]:
xrays = pd.read_csv('../data/xrays-eda-clean.csv')

In [ ]:
# show index number for each column
# pd.DataFrame({'cols':xrays.columns.tolist()})

In [ ]:
xrays.head()

In [ ]:
xrays.columns.tolist()

In [ ]:
# inspect column data types and look for null values
xrays.info()

## 3. Begin EDA

### Check basic stats for numeric variables

In [ ]:
# show basic stats for any numeric columns
xrays.describe()

**How many x-rays are there? How many are normal vs. non-normal?**

In [ ]:
n = xrays[xrays['Finding Labels']=='No Finding']['Image Index'].count()
nn = xrays.shape[0] - n
print("Total X-rays: \t{:,}".format(n + nn))
print("- Normal: \t {:,}".format(n))
print("- Non-normal: \t {:,}".format(nn))

In [ ]:
# increase font label size
sns.set_context("notebook", font_scale=1.25, rc={"lines.linewidth": 2.5})

# Plot normal v. non-normal X-ray counts
x = xrays['Non-normal']
ax = sns.countplot(x);

# label bars with counts
for p, label in zip(ax.patches, x.value_counts()):
    ax.annotate(label, (p.get_x()+0.3, p.get_height()+0.5))

plt.title('Normal vs. Non-normal X-rays')

# save PNG
plt.savefig('charts/countplot_normal_nonnormal.png', bbox_inches='tight')

**How many patients are there?**

In [ ]:
# Count unique 'Patient ID'
p = xrays['Patient ID'].nunique()
print("Total Patients: {:,}".format(p))

puniqnonnorm = xrays[xrays['Non-normal'] == 1]['Patient ID'].nunique()
print("- Patients with Non-normal X-rays: {:,}".format(puniqnonnorm))

puniqnorm = xrays[xrays['Non-normal'] == 0]['Patient ID'].nunique()
print("- Patients with Normal X-rays: {:,}".format(puniqnorm))

Observation: Many patients who have multiple X-rays must have both normal and non-normal results.

**How many of the patients with multiple X-rays have both normal and non-normal findings among those X-rays?**

In [ ]:
# def fun_pcheck(p, df):
#     '''Returns True if patient has both normal and non-normal X-rays'''
#     patient = df[df['Patient ID'] == p]    
#     if (len(patient['Non-normal'].unique()) == 1):
#         return False # X-rays are all normal or all non-normal
#     return True # X-rays have mixed results

# IDs of patients with multiple X-rays
# pids_multi = xrays[xrays['Follow-up #'] > 0]['Patient ID'].unique()

# subset columns
# cols = ['Patient ID','Non-normal']
# sub1 = xrays[cols]

# count of patients with combo results (norm/non)
# pboth = 0
# for p in pids_multi:    
#     if (fun_pcheck(p, sub1) == True):
#        pboth += 1

# print('There are {:,} patients with more than one X-ray.'.format(len(pids_multi)))
# print('Of these patients, {:,} have both normal and non-normal X-rays.'.format(pboth))

There are 13,302 patients with more than one X-ray.  
Of these patients, 8,504 have both normal and non-normal X-rays.

In [ ]:
xrays['Age Group'].value_counts(sort=False)

In [ ]:
df = xrays[['Age Group']].sort_values(by=['Age Group'])
g = sns.countplot(df['Age Group'], color='cadetblue');
plt.setp(g.get_xticklabels(), rotation=60);
plt.title('Patient Age');

# save chart as PNG
plt.savefig('charts/age_groups.png', bbox_inches='tight');

**As of the first x-ray, what are patients' ages?**

In [ ]:
# first X-ray
xr1 = xrays[xrays['Follow-up #'] == 0]
print("The plot below represents all {:,} patients.".format(xr1.shape[0]))

A benefit of only looking at the first X-ray is that each patient is counted once, as opposed to once per X-ray.

In [ ]:
# Enlarge the plot
plt.figure(figsize=(3.5,3));

# plot code adapted from sbernadac (Kaggle) to address the "first X-ray" question
sns.set(font_scale = 1.5);
g = sns.factorplot(x="Patient Age", col="Patient Gender",data=xr1, kind="count",size=10, aspect=0.8,palette="husl");
g.set_xticklabels(np.arange(0,100));
g.set_xticklabels(step=10);
g.fig.suptitle('Patient Age by Gender using 30,805 First X-rays,\nwhere Follow-up # is Zero',fontsize=22);
g.fig.subplots_adjust(top=.9);

# save chart as PNG
plt.savefig('charts/age_by_gender_first_xray.png', bbox_inches='tight');

In [ ]:
# count M and F in first xray
totalmen = xr1[xr1['Patient Gender'] == 'M']['Patient ID'].nunique()
totalmenxrays = xrays[xrays['Patient Gender'] == 'M']['Image Index'].nunique()
totalwommen = xr1[xr1['Patient Gender'] == 'F']['Patient ID'].nunique()
totalwomenxrays = xrays[xrays['Patient Gender'] == 'F']['Image Index'].nunique()
print('The dataset contains:')
print( '- {:,} X-rays from {:,} men.'.format(totalmenxrays, totalmen))
print( '- {:,} X-rays from {:,} wommen.'.format(totalwomenxrays, totalwommen))

In [ ]:
# plot code adapted from sbernadac (Kaggle)
sns.set(font_scale = 1.5)
g = sns.factorplot(x="Patient Age", col="Patient Gender",data=xrays, kind="count",size=10, aspect=0.8,palette="husl");
g.set_xticklabels(np.arange(0,100));
g.set_xticklabels(step=10);
g.fig.suptitle('Patient Age by Gender using All 112,120 X-rays',fontsize=22);
g.fig.subplots_adjust(top=.9)

# save chart as PNG
plt.savefig('charts/age_by_gender_all_xrays.png', bbox_inches='tight')

### Finding Labels
**What findings are most or least common in the X-rays?**

In [ ]:
# pathology counts
pathos = xrays.iloc[:,11:25].sum().sort_values(ascending=False)
g = sns.barplot(x=pathos, y=pathos.index);

In [ ]:
print(pathos)

### Correlation: Among non-normal X-ray finding labels, do any correlations exist?

In [ ]:
# Enlarge the plot
plt.figure(figsize=(7.5,7))

# index of non-normal rows
idx_non_normals = xrays[xrays['Non-normal'] == 1].index

# Calculate correlations
corr = xrays.iloc[idx_non_normals, 11:27].corr()

# Heatmap
sns.heatmap(corr, cmap='PiYG', center=0);

# save PNG
plt.savefig('charts/corr_diseases.png', bbox_inches='tight')

Insights from pair-wise correlation plots:
- Overall negative correlations reflects that finding typically occur alone or in small numbers.
- The strongest among negative correlations are:
    - Negative correlation of -0.143 between Infiltration and Pneumothorax
    - Negative correlation of -0.134 between Infiltration and Mass
    - Negative correlation of -0.117 between Atelectasis and Nodule
    - Negative correlation of -0.102 between Infiltration and Effusion
- The strongest among positive correlations are:
    - Positive correlation of 0.158 between Pneumonia and Edema
    - Positive correlation of 0.145 between Emphysema and Pneumothorax
    - Positive correlation of 0.068 between Cardiomegaly and Effusion
- Correlations with TotalPathos relates to total counts per [Finding Labels](#Finding-Labels)
- Total pathologies are most positively correlated with Effusion, Atelectasis, and Infiltration
- Total pathologies are least correlated with Hernia and Fibrosis

In [ ]:
# display correlations dataframe
corr

### Correlation: Any patterns in this mixed bag of numeric variables?

In [ ]:
# Enlarge the plot
plt.figure(figsize=(5.5,5))

# Calculate correlations
corr2 = xrays.iloc[:, [2,3,7,8,9,10,25,26]].corr()

# Heatmap
sns.heatmap(corr2, cmap='PiYG');

# save PNG
plt.savefig('charts/corr_img_oddends.png', bbox_inches='tight')

Observations and oddities:
- Image Width and Height have a strong *negative* correlation with Image Pixel Spacing--a technical side note.
- Negative correlation of follow-up # and image height; this might be an anomaly of outlier leverage by a few images with high follow-up # that happen to be shorter images
- Patient ID is negatively correlated with Image Pixel Spacing; this could reflect serialized dataset aggreation from sources with different standard sizes

**Is there any correlation between Age and Follow-up #?**

In [ ]:
# Avg. Patient Age as of the 11th X-ray
xrays[xrays['Follow-up #'] == 10]['Patient Age'].mean(axis=0)

In [ ]:
# Compute average age of patients at each X-ray Follow-up #
maxfu = xrays['Follow-up #'].max(axis=0) # 183
fu = []   # follow-up number
favage = [] # avg. age
for f in range(maxfu):
    a = xrays[xrays['Follow-up #'] == f]['Patient Age'].mean(axis=0)
    fu.append(f)
    favage.append(a)

df_fufav = pd.DataFrame({'Follow-up #':fu, 'Average Age':favage})

In [ ]:
# Plot Average Age by Follow-up #
g = sns.lmplot(x='Follow-up #', y='Average Age', data=df_fufav);
plt.title('Average Age by Follow-up #');

In [ ]:
# plot with simple linear regression
sns.lmplot(x='Follow-up #', y='Patient Age', data=xrays,
           fit_reg=False, # No regression line
           hue='Patient Gender');

Observations:
* Follow-up # ranges from low of 0 (first visit) to high of 183 with a median of 3
* Patient Age ranges from 1 to 414; need to investigate extreme values
* 

Next, explore the variety of image widths, heights, and pixel spacing.

### View Position: Posterior-Anterior(PA) / Anterior-Posterior (AP)

In [ ]:
# List categories and counts
for cat in ['Patient Gender', 'View Position']:
    print( "{}\n".format(xrays[cat].value_counts()) )

### X-ray image size and dimensions

In [ ]:
w = len(xrays["ImgWidth"].value_counts())
h = len(xrays["ImgHeight"].value_counts())
print("Unique image widths: {}\nUnique image heights: {}".format(w, h))

In [ ]:
# create column with combined image width by height string
xrays.loc[: ,"ImageWxH"] = xrays["ImgWidth"].map(str) + "x" + xrays["ImgHeight"].map(str)

In [ ]:
xrays["ImageWxH"].head()

In [ ]:
# summarize unique combinations of image dimensions
wh = len(xrays["ImageWxH"].value_counts())
print("Number of unique image dimensions (WxH) among chest X-rays: {}".format(wh))

In [ ]:
xrays["ImgWidth"].sort_values().tail()

### X-ray Image Resolution in Megapixels

In [ ]:
# Enlarge the plot
plt.figure(figsize=(7.5,5))

# histogram of image W x H
sns.distplot(xrays["ImgWidth"]*xrays["ImgHeight"]/1000000, bins=22,  kde=False);
plt.title('Histogram of X-ray Image Resolution');
plt.xlabel('Megapixels');
plt.ylabel('X-ray counts');

# save PNG
plt.savefig('charts/hist_total_pixels.png', bbox_inches='tight')

In [ ]:
# most common image dimensions
xrays["ImageWxH"].value_counts().head(10)

In [ ]:
x = len(xrays["ImgPxSpaceX"].value_counts())
y = len(xrays["ImgPxSpaceY"].value_counts())
print("Unique x pixel spacings: {}\nUnique y pixel spacings: {}".format(x, y))

### Outliers for Patient Age

In [ ]:
# histogram by Age
sns.distplot(xrays["Patient Age"], bins=25,  kde=False);
plt.title('Histogram of Patient Age with Full Data');

**What are the outliers at high end of 'Patient Age'?**

In [ ]:
xrays.iloc[:, 0:6].sort_values(by='Patient Age', ascending=False).head(20)

Finding: 16 X-rays list Patient Age of 148 or greater. Perhaps that value is mislabeled, or not 'years' in some instances. Look at same patient where possible.

In [ ]:
# Filter by 'Patient ID' where 'Patient Age' is over 100
xrays[xrays['Patient ID'].isin(xrays[xrays['Patient Age'] > 100]['Patient ID'])].tail(10)

Findings:
- Patient ID 25206 is Age 153 at Follow-up \#0, then Age 36 at all future Follow-ups.
- Patient ID 26028 is Age 60 at Follow-up \#0, but Age 154 at future Follow-ups.

**Age Outliers Conclusion:** A handful of 'Patient Age' outliers reveal errors in this data. In cases where an Age outlier Patient ID has matching Follow-up #'s, the Patient Age could be corrected on a case-by-case basis. This could improve EDA statistical summaries, or if Age were to be used as a data modeling input. Regardless, the rows for these X-ray images will be left in the data set.


In [ ]:
# histogram by Age < 100
sns.distplot(xrays[xrays["Patient Age"] < 100]["Patient Age"], bins=19,  kde=False);
plt.title('Histogram of Patient Age < 100');

### Outliers for Follow-up #
**What is the range and distribution of "Follow-up #"?**

In [ ]:
# quick stats on multiple series
xrays[["TotalPathos", "Follow-up #"]].describe()

In [ ]:
# List top 20 X-ray counts by Follow-up #
a = xrays['Follow-up #']
unique_elements, counts_elements = np.unique(a, return_counts=True)

# counts_elements_proportion_of_patients
counts_elements_pop = counts_elements/counts_elements[0]
print("Frequency of unique values of the said array:")
#print(np.asarray((unique_elements, counts_elements)))
xrfu = pd.DataFrame( OrderedDict( {'Follow-up #':unique_elements, 'X-ray count':counts_elements, 'Proportion of patients':counts_elements_pop} ) )
xrfu.head(20)

In [ ]:
xrfu[30:36]

**Insights based on Follow-up #:**
- More than 90% of patients have fewer than 10 X-rays
- Fewer than 1% of patients have more than 35 X-rays

In [ ]:
x = xrays['Follow-up #']
plt.figure(figsize=(16, 9))
g = sns.countplot(x);
g.set_xticklabels(np.arange(min(x), max(x)+1, step=1));
#g.set_xticklabels(step=10);
plt.setp(g.get_xticklabels(), rotation=90);
plt.title('Count of X-rays by Follow-up #');

Insights:
* A given x-ray has anywhere from 0 to 9 total pathologies.
* The "Follow-up #" ranges from 0 (first visit and x-ray) to 183.

### Outliers for Multi-label Findings in an X-ray

In [ ]:
x = xrays['TotalPathos']
sns.countplot(x);
plt.title('Count of X-rays by total number of pathologies found');

In [ ]:
# Plot of "Follow-up #" with 
sns.jointplot(x="Follow-up #", y="TotalPathos", data=xrays);

### Example of Single Patient X-ray History

In [ ]:
xrays[xrays['Patient ID'] == 11997]

# References

Where noted above, select charts and code wizardry are credited to the excellent [Lung diseases data analysis](https://www.kaggle.com/sbernadac/lung-deseases-data-analysis) by Stephane Bernadec (sbernadec) on Kaggle.